In [2]:
# %pip install mysql-connector-python

In [4]:
import mysql.connector

# Establish connection
con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="pdbc"
)

# Create a cursor object
cursor = con.cursor()


In [5]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100),
        age INT,
        department VARCHAR(50)
    )
""")
con.commit()


In [6]:
sql = "INSERT INTO employees (name, age, department) VALUES (%s, %s, %s)"
values = [("Arjun", 30, "HR"), ("Drishya", 28, "IT"), ("Krishna", 35, "Finance")]

cursor.executemany(sql, values)
con.commit()
print(cursor.rowcount, "records inserted.")


3 records inserted.


In [7]:
cursor.execute("SELECT * FROM employees")
rows = cursor.fetchall()

for row in rows:
    print(row)  # Output each row


(1, 'Arjun', 30, 'HR')
(2, 'Drishya', 28, 'IT')
(3, 'Krishna', 35, 'Finance')


In [10]:
sql = "UPDATE employees SET department = %s WHERE name = %s"
values = ("Marketing", "Arjun")

cursor.execute(sql, values)
con.commit()
print(cursor.rowcount, "record updated.")


1 record updated.


In [11]:
cursor.execute("SELECT * FROM employees")
rows = cursor.fetchall()

for row in rows:
    print(row) 

(1, 'Arjun', 30, 'Marketing')
(2, 'Drishya', 28, 'IT')
(3, 'Krishna', 35, 'Finance')


In [13]:
sql = "DELETE FROM employees WHERE name = %s"
values = ("Arjun",)

cursor.execute(sql, values)
con.commit()
print(cursor.rowcount, "record deleted.")


1 record deleted.


In [14]:
cursor.execute("SELECT * FROM employees")
rows = cursor.fetchall()

for row in rows:
    print(row) 

(2, 'Drishya', 28, 'IT')
(3, 'Krishna', 35, 'Finance')


In [15]:
cursor.close()
con.close()


In [17]:
import mysql.connector
from mysql.connector import Error

# Database connection function
def create_connection():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="root",
            database="pdbc"
        )
        if conn.is_connected():
            print("Connected to MySQL database")
        return conn
    except Error as e:
        print(f"Error: {e}")
        return None

# Function to create table
def create_table():
    conn = create_connection()
    if conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS employees (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(100) NOT NULL,
                    age INT NOT NULL,
                    department VARCHAR(50) NOT NULL
                )
            """)
        conn.commit()
        conn.close()

# Function to insert a record
def insert_employee(name, age, department):
    conn = create_connection()
    if conn:
        with conn.cursor() as cursor:
            sql = "INSERT INTO employees (name, age, department) VALUES (%s, %s, %s)"
            cursor.execute(sql, (name, age, department))
        conn.commit()
        print("Employee inserted successfully!")
        conn.close()

# Function to read all employees
def fetch_employees():
    conn = create_connection()
    if conn:
        with conn.cursor() as cursor:
            cursor.execute("SELECT * FROM employees")
            rows = cursor.fetchall()
            for row in rows:
                print(row)
        conn.close()

# Function to update an employee's department
def update_employee_department(emp_id, new_department):
    conn = create_connection()
    if conn:
        with conn.cursor() as cursor:
            sql = "UPDATE employees SET department = %s WHERE id = %s"
            cursor.execute(sql, (new_department, emp_id))
        conn.commit()
        print("Employee updated successfully!")
        conn.close()

# Function to delete an employee
def delete_employee(emp_id):
    conn = create_connection()
    if conn:
        with conn.cursor() as cursor:
            sql = "DELETE FROM employees WHERE id = %s"
            cursor.execute(sql, (emp_id,))
        conn.commit()
        print("Employee deleted successfully!")
        conn.close()

# Running the functions
if __name__ == "__main__":
    create_table()
    
    # Insert sample data
    insert_employee("Avi", 30, "HR")
    insert_employee("bhuvi", 28, "IT")
    insert_employee("Charu", 35, "Finance")

    # Fetch and display all employees
    print("\nAll Employees:")
    fetch_employees()

    # Update an employee's department
    update_employee_department(2, "Marketing")

    # Fetch updated employees
    print("\nEmployees after update:")
    fetch_employees()

    # Delete an employee
    delete_employee(3)

    # Fetch final list of employees
    print("\nEmployees after deletion:")
    fetch_employees()


Connected to MySQL database
Connected to MySQL database
Employee inserted successfully!
Connected to MySQL database
Employee inserted successfully!
Connected to MySQL database
Employee inserted successfully!

All Employees:
Connected to MySQL database
(2, 'Drishya', 28, 'IT')
(3, 'Krishna', 35, 'Finance')
(4, 'Avi', 30, 'HR')
(5, 'bhuvi', 28, 'IT')
(6, 'Charu', 35, 'Finance')
Connected to MySQL database
Employee updated successfully!

Employees after update:
Connected to MySQL database
(2, 'Drishya', 28, 'Marketing')
(3, 'Krishna', 35, 'Finance')
(4, 'Avi', 30, 'HR')
(5, 'bhuvi', 28, 'IT')
(6, 'Charu', 35, 'Finance')
Connected to MySQL database
Employee deleted successfully!

Employees after deletion:
Connected to MySQL database
(2, 'Drishya', 28, 'Marketing')
(4, 'Avi', 30, 'HR')
(5, 'bhuvi', 28, 'IT')
(6, 'Charu', 35, 'Finance')


In [19]:
# %pip install pymongo


In [25]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

# Function to establish a MongoDB connection
def create_connection():
    try:
        client = MongoClient("mongodb://localhost:27017/")
        db = client["pdbc"]
        print("Connected to MongoDB successfully!")
        return db
    except ConnectionFailure as e:
        print(f"Connection error: {e}")
        return None

# Function to create a collection (if it doesn’t exist)
def create_collection():
    db = create_connection()
    if db is not None:
        db.create_collection("employees", capped=False)  # capped=True limits size
        print("Collection created (if not already exists).")

# Function to insert a single document
def insert_employee(name, age, department):
    db = create_connection()
    if db is not None:
        employees = db["employees"]
        emp_data = {"name": name, "age": age, "department": department}
        employees.insert_one(emp_data)
        print("Employee inserted successfully!")

# Function to insert multiple employees
def insert_multiple_employees():
    db = create_connection()
    if db is not None:
        employees = db["employees"]
        bulk_data = [
            {"name": "Varun", "age": 30, "department": "HR"},
            {"name": "karna", "age": 28, "department": "IT"},
            {"name": "Rohit", "age": 35, "department": "Finance"}
        ]
        employees.insert_many(bulk_data)
        print("Multiple employees inserted successfully!")

# Function to read all employees
def fetch_employees():
    db = create_connection()
    if db is not None:
        employees = db["employees"]
        for emp in employees.find():
            print(emp)

# Function to update an employee's department
def update_employee_department(name, new_department):
    db = create_connection()
    if db is not None:
        employees = db["employees"]
        result = employees.update_one({"name": name}, {"$set": {"department": new_department}})
        if result.modified_count > 0:
            print("Employee updated successfully!")
        else:
            print("No employee found with that name.")

# Function to delete an employee
def delete_employee(name):
    db = create_connection()
    if db is not None:
        employees = db["employees"]
        result = employees.delete_one({"name": name})
        if result.deleted_count > 0:
            print("Employee deleted successfully!")
        else:
            print("No employee found with that name.")

# Running CRUD operations
if __name__ == "__main__":
    # create_collection()

    # Insert sample data
    insert_employee("vinita", 30, "HR")
    insert_multiple_employees()

    # Fetch and display all employees
    print("\nAll Employees:")
    fetch_employees()

    # Update an employee's department
    update_employee_department("Viswa", "Marketing")

    # Fetch updated employees
    print("\nEmployees after update:")
    fetch_employees()

    # Delete an employee
    delete_employee("drishya")

    # Fetch final list of employees
    print("\nEmployees after deletion:")
    fetch_employees()


Connected to MongoDB successfully!
Employee inserted successfully!
Connected to MongoDB successfully!
Multiple employees inserted successfully!

All Employees:
Connected to MongoDB successfully!
{'_id': ObjectId('67a21177e75496d0edb240be'), 'name': 'vinita', 'age': 30, 'department': 'HR'}
{'_id': ObjectId('67a21177e75496d0edb240c0'), 'name': 'Varun', 'age': 30, 'department': 'HR'}
{'_id': ObjectId('67a21177e75496d0edb240c1'), 'name': 'karna', 'age': 28, 'department': 'IT'}
{'_id': ObjectId('67a21177e75496d0edb240c2'), 'name': 'Rohit', 'age': 35, 'department': 'Finance'}
Connected to MongoDB successfully!
No employee found with that name.

Employees after update:
Connected to MongoDB successfully!
{'_id': ObjectId('67a21177e75496d0edb240be'), 'name': 'vinita', 'age': 30, 'department': 'HR'}
{'_id': ObjectId('67a21177e75496d0edb240c0'), 'name': 'Varun', 'age': 30, 'department': 'HR'}
{'_id': ObjectId('67a21177e75496d0edb240c1'), 'name': 'karna', 'age': 28, 'department': 'IT'}
{'_id': Obj